In [1]:
# | default_exp routes.datacenter

In [2]:
# | exporti
from enum import Enum

import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

In [3]:
#| export
class Datacenter_Enum(Enum):
    DATASET = "DATASET"
    DATAFLOW = "DATAFLOW"
    ACCOUNT = "ACCOUNT"


class Dataflow_Type_Filter_Enum(Enum):
    ADR = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "ADR",
        "name": "ADR",
        "not": False,
    }

    MYSQL = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "MYSQL",
        "name": "MYSQL",
        "not": False,
    }

    REDSHIFT = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "MYSQL",
        "name": "MYSQL",
        "not": False,
    }

    MAGICV2 = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "MAGIC",
        "name": "Magic ETL v2",
        "not": False,
    }

    MAGIC = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "ETL",
        "name": "Magic ETL",
        "not": False,
    }

In [4]:
# | export
def generate_search_datacenter_body(
    entities_list: list[str] = None, # defaults to Dataset
    filters: list[dict] = None,
    combineResults: bool = True,
    count: int = 10,
    offset: int = 0,
):
    entities_list = entities_list or ["DATASET"]
    return {
        "entities": entities_list,
        "filters": filters or [],
        "combineResults": combineResults,
        "query": "*",
        "count": count,
        "offset": offset,
    }

In [5]:
# | export
def generate_search_datacenter_account_body(
    search_str: str, is_exact_match: bool = True
):
    return {
        "count": 100,
        "offset": 0,
        "combineResults": False,
        "query": search_str if is_exact_match else f"*{search_str}*",
        "filters": [],
        "facetValuesToInclude": [
            "DATAPROVIDERNAME",
            "OWNED_BY_ID",
            "VALID",
            "USED",
            "LAST_MODIFIED_DATE",
        ],
        "queryProfile": "GLOBAL",
        "entityList": [["account"]],
        "sort": {"fieldSorts": [{"field": "display_name_sort", "sortOrder": "ASC"}]},
    }

In [6]:
# | export
async def search_datacenter(
    auth: dmda.DomoAuth,
    loop_until_end: bool = True,
    maximum: int = None,
    body: dict = None,
    entity_ls: list[str] = None, # defaults to dataset
    arr_fn: callable = None,
    limit=100,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    entity_ls = entity_ls or ["DATASET"]
    if not body:
        body = generate_search_datacenter_body(
            entities_list=entity_ls,
            filters=[],
            combineResults=False,
        )

    if not arr_fn:

        def arr_fn(res):
            return res.response.get("searchObjects")

    url = f"https://{auth.domo_instance}.domo.com/api/search/v1/query"

    res = await gd.looper(
        auth=auth,
        session=session,
        url=url,
        body=body,
        offset_params={"offset": "offset", "limit": "count"},
        arr_fn=arr_fn,
        method="POST",
        maximum = maximum,
        offset_params_in_body=True,
        limit=limit,
        loop_until_end=loop_until_end,
        debug_api=debug_api,
    )

    return res

#### sample implementation of generate_search body and search datacenter


In [7]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

body = generate_search_datacenter_body(
    entities_list=[Datacenter_Enum.DATAFLOW.value],
    filters=[Dataflow_Type_Filter_Enum.MAGICV2.value],
)

res = await search_datacenter(auth=token_auth, body=body)

pd.DataFrame(res.response[0:5])

,entityType,databaseId,searchId,createDate,lastModified,lastIndexed,highlightedFields,language,requestAccess,score,...,abandoned,owners,ownersLocalized,paused,winnerText,statusOrderPriority,inputCount,outputCount,ownedByType,customer
0,dataflow,131,"{'indexName': None, 'databaseId': '131', 'cust...",1678147044000,1678147835000,1680574033100,{},English,False,10.000001,...,False,"[{'id': '1893952720', 'type': 'USER', 'display...","{'localizedMessage': 'Jae Wilson1', 'count': 1}",False,BeastModesGovernance_01_AccumulateHistory,70,1,1,USER,mmmm-0012-0200
1,dataflow,37,"{'indexName': None, 'databaseId': '37', 'custo...",1619005428000,1619007232000,1619080673093,{},English,False,10.000001,...,False,"[{'id': '1893952720', 'type': 'USER', 'display...","{'localizedMessage': 'Jae Wilson', 'count': 1}",False,EX01_FuzzyMatch_Master,70,2,1,USER,mmmm-0012-0200
2,dataflow,124,"{'indexName': None, 'databaseId': '124', 'cust...",1675787096000,1675787978000,1675789364742,{},English,False,10.000001,...,False,"[{'id': '1893952720', 'type': 'USER', 'display...","{'localizedMessage': 'Jae Wilson', 'count': 1}",False,Recursive+Partition_Demo,70,1,1,USER,mmmm-0012-0200
3,dataflow,38,"{'indexName': None, 'databaseId': '38', 'custo...",1619005563000,1619077579000,1619080673093,{},English,False,10.000001,...,False,"[{'id': '1893952720', 'type': 'USER', 'display...","{'localizedMessage': 'Jae Wilson', 'count': 1}",False,EX01_FuzzyMatch_Master_RECURSIVE,70,3,1,USER,mmmm-0012-0200
4,dataflow,46,"{'indexName': None, 'databaseId': '46', 'custo...",1620395295000,1620395329000,1620630957077,{},English,False,9.000001,...,False,"[{'id': '1251168784', 'type': 'USER', 'display...","{'localizedMessage': 'Andre Wieczorkiewicz', '...",False,AW_delete,70,1,1,USER,mmmm-0012-0200


In [8]:
# | export
async def get_lineage_upstream(
    auth: dmda.DomoAuth,
    entity_type: str,
    entity_id: str,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
):
    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/lineage/{entity_type}/{entity_id}"

    params = {"traverseDown": "false"}

    return await gd.get_data(
        auth=auth,
        method="GET",
        url=url,
        params=params,
        session=session,
        debug_api=debug_api,
    )

In [9]:
import os
# import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance='domo-community',
    domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

res = await get_lineage_upstream(auth=token_auth,
                                 entity_type = Datacenter_Enum.DATAFLOW.value,
                                 entity_id=4

                                 )

pd.DataFrame([ obj for obj in res.response.values()])

,type,id,descendantCounts,ancestorCounts,complete,children,parents
0,DATA_SOURCE,45a7a24e-c738-44f9-9019-00d5a33613ae,{},"{'DATA_SOURCE': 2, 'DATAFLOW': 2}",True,"[{'type': 'DATAFLOW', 'id': '4', 'complete': T...","[{'type': 'DATAFLOW', 'id': '3', 'complete': T..."
1,DATAFLOW,4,{},"{'DATA_SOURCE': 3, 'DATAFLOW': 2}",True,[],"[{'type': 'DATA_SOURCE', 'id': '45a7a24e-c738-..."
2,DATAFLOW,2,{},{'DATA_SOURCE': 1},True,"[{'type': 'DATA_SOURCE', 'id': '0647a9ed-1a8e-...","[{'type': 'DATA_SOURCE', 'id': '241025d7-3cca-..."
3,DATA_SOURCE,0647a9ed-1a8e-420e-bd66-34f313a18595,{},"{'DATA_SOURCE': 1, 'DATAFLOW': 1}",True,"[{'type': 'DATAFLOW', 'id': '3', 'complete': T...","[{'type': 'DATAFLOW', 'id': '2', 'complete': T..."
4,DATAFLOW,3,{},"{'DATA_SOURCE': 2, 'DATAFLOW': 1}",True,"[{'type': 'DATA_SOURCE', 'id': '45a7a24e-c738-...","[{'type': 'DATA_SOURCE', 'id': '0647a9ed-1a8e-..."
5,DATA_SOURCE,241025d7-3cca-4369-b7c0-b3264277c0e1,{},{},True,"[{'type': 'DATAFLOW', 'id': '2', 'complete': T...",[]


In [10]:
# | hide
import nbdev

nbdev.nbdev_export()
